# Tangent actuators No back driving analysis

This notebook analyzes the NO back driving data of M2 tangent links. 

The test has been performed with the configuration file: **ts_mtm2/config/sysconfig
/Configurable_File_Description_20180831T092556_surrogate_handling.csv**.

The goal is to verify that:

1. The tangent actuators do no back drive when the M2 cell is powered off
2. The M2 mirror does not drift in position when the power is cut.

Link to the test case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1784

## Import Modules

This notebook needs to setup the **ts_m2com** and **ts_aos_utilsts** under the **notebooks/.user_setups**, which depends on the **ts_tcpip**.
You also need to have **ts_mtm2** under the **WORK/** directory to read the confiugration files to do the analysis.

In [ ]:
%matplotlib inline
%matplotlib widget  
import matplotlib.pyplot as plt
import math
from pathlib import Path
import numpy as np
from sklearn.linear_model import LinearRegression
from astropy.time import Time
import pandas as pd
import lsst_efd_client

from lsst.ts.m2com import MockControlClosedLoop
import tabulate 
from lsst.ts.aos.utils import DiagnosticsM2, EfdName

## Functions declaration

In [ ]:
# function to retrieve EFD topics

async def query_data(
    diagnostics_m2: DiagnosticsM2,
    control_closed_loop: MockControlClosedLoop,
    time_start: Time,
    time_end: Time,
) -> None:
    """
    Query the data.

    # functions declaration can be found here:
    # https://github.com/lsst-ts/ts_aos_utils/blob/feature/m2power/python/lsst/ts/aos/utils/diagnostics_m2.py
    Parameters
    ----------
    diagnostics_m2 : `lsst.ts.aos.utils.DiagnosticsM2`
        M2 diagnostics instance.
    control_closed_loop : `lsst.ts.m2.com.MockControlClosedLoop`
        Mock control closed loop instance.
    time_start : `astropy.time.core.Time`
        Start time.
    time_end : `astropy.time.core.Time`
        End time.
    """

    # Get the x, y position of actuators
    xy_actuators = diagnostics_m2.get_xy_actuators(control_closed_loop)

    # Query data
    data_ims, time_operation_ims = await diagnostics_m2.get_data_position(
        "positionIMS", time_start, time_end
    )

    data_collected_axial, data_collected_tangent = await diagnostics_m2.get_data_force(
        time_start, time_end
    )

    data_power_status, time_operation = await diagnostics_m2.get_data_power_status(
        time_start, time_end
    )

    data_inclinometer = await diagnostics_m2.get_data_zenith_angle(
        time_start, time_end
    )

    return xy_actuators, data_ims, data_collected_axial, data_collected_tangent, data_power_status, data_inclinometer

In [ ]:
# function that prints variable values at terminal for sanity check

def print_data(
    data_collected_axial, data_collected_tangent, data_inclinometer, data_ims, data_power_status, actuators_groups, group, face
):
    # number elements force vector
    # convert tuple into integer
    temp = np.shape(data_collected_tangent["measured"][:, 0])
    y = "".join(map(str, temp))
    n_elem_force = int(y)

    # time interval data stream
    delta_t = data_ims.index[-1] - data_ims.index[0]

    print(data_ims.index[0])
    print(data_ims.index[-1])
    print(f"Signal length, sec: {delta_t.seconds}")

    fig = plt.figure()
    font = {
        "family": "serif",
        "color": "black",
        "weight": "normal",
        "size": 10,
    }
    ax = fig.add_subplot(111)
    ax.plot(data_power_status.index, data_power_status.motorCurrent)
    ax.set_xlabel("UTC", fontdict=font)
    ax.set_ylabel("Motor current, A", fontdict=font)

    fig.tight_layout()


    
    
    # array number of elements
    n_elem_motorcurrent = len(data_power_status.motorCurrent)
    n_elem_ims = len(data_ims.x)
    n_elem_inclinometer = len(data_inclinometer["inclinometerProcessed"])

    # scaling factors
    relative_scales = n_elem_force / n_elem_motorcurrent
    relative_scales_ims = n_elem_ims / n_elem_motorcurrent
    relative_scales_ims_force = n_elem_force / n_elem_ims
    relative_scales_inclinometer_force = n_elem_force / n_elem_inclinometer
 

    print(f"CHECKS dimensions:")
    full_stream = len(data_collected_tangent["measured"])
    print(f"force length: {full_stream}")
    print(f"ims length: {n_elem_ims}")
    print(f"motor current length: {n_elem_motorcurrent}")
    print(f"inclinometer length: {n_elem_inclinometer}")

    print(f"force / motor cur: {relative_scales}")
    print(f"ims / motor cur: {relative_scales_ims}")
    print(f"force / ims: {relative_scales_ims_force}")
    print(f"force / inclinometer: {relative_scales_inclinometer_force}")
    
    
    # find the time M2 cart reaches the vertical position
    for idx in range(len(data_inclinometer["inclinometerProcessed"])):

        if abs(data_inclinometer["inclinometerProcessed"].iloc[idx] - 1) < 1:
            position_reached_time = idx
            break
            
        elif abs(180 + data_inclinometer["inclinometerProcessed"].iloc[idx]) < 0.3:
            position_reached_time = idx
            break
    
    
    
    # find time IMS start
    m2_in_position = np.where(data_ims.index >= data_inclinometer.index[idx])[0] 
    start_time_ims = m2_in_position[0]
    start_time_encoder = data_ims.index[m2_in_position[0]]

    
    print(f"M2 in vertical position, time inclinometer: {data_inclinometer.index[idx]}")
    print(f"IMS start, time IMS: {data_ims.index[m2_in_position[0]]}")
    print(f"M2 reaches vertical position, index inclinometer: {position_reached_time}")

    # M2 vertical position reached in force index metrics
    position_reached_index_force = round(position_reached_time*relative_scales_inclinometer_force)
    print(f"M2 reaches vertical position, index force: {position_reached_index_force}")


    
    # compensate for lack of telemetry during A2-A6 E-stop data acquisition 
    if face == 'up':
        position_reached_index_force = round(position_reached_index_force*1.3)
 
    
    # find time of Estop push
    estop_push = np.where(data_power_status.motorCurrent == 0)[0]
    time_estop = data_power_status.index[estop_push[0]]

    eStop = round(estop_push[0] * relative_scales)
    eStop_ims = round(estop_push[0] * relative_scales_ims) + 10
    print(f"E-stop push time: {eStop}")

    # number of data points in the interval
    full_stream = len(data_collected_tangent["measured"])
    print(f"Force data points: {full_stream}")


    # **************************************
    #
    #  MEASURED FORCES after Estop push
    #
    # **************************************

    
    statistics_pre_estop = np.zeros((6, 6))
    statistics_post_estop = np.zeros((6, 6))


    n = 6
    for idx in range(n):
        print(f"actuator: {actuators_groups[idx]+1}")


        # remove the average value from each signal
        avg_signal_pre = (
            data_collected_tangent["measured"][position_reached_index_force:eStop, idx]
            - data_collected_tangent["measured"][position_reached_index_force:eStop, idx].mean()
        )

        avg_signal_post = (
            data_collected_tangent["measured"][eStop:full_stream, idx]
            - data_collected_tangent["measured"][eStop:full_stream, idx].mean()
        )


        
        # PRE
        statistics_pre_estop[0, idx] = avg_signal_pre.max()
        statistics_pre_estop[1, idx] = avg_signal_pre.min()
        statistics_pre_estop[2, idx] = avg_signal_pre.max() - avg_signal_pre.min()
        statistics_pre_estop[4, idx] = math.sqrt(
            sum((avg_signal_pre) ** 2 / len(avg_signal_pre))
        )

        # POST
        statistics_post_estop[0, idx] = avg_signal_post.max()
        statistics_post_estop[1, idx] = avg_signal_post.min()
        statistics_post_estop[2, idx] = avg_signal_post.max() - avg_signal_post.min()
        statistics_post_estop[4, idx] = math.sqrt(
            sum((avg_signal_post) ** 2 / len(avg_signal_post))
        )

    return (
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        eStop,
        eStop_ims,
        time_estop,
        position_reached_index_force,  
        start_time_ims,
        start_time_encoder,
    )

In [ ]:
# function that plots actuator forces

def actuator_plot(
    diagnostics_m2,
    control_closed_loop,
    position_reached_index_force,
    actuators_groups,
    group,
    eStop,
    data_collected_axial,
    data_collected_tangent,
    full_stream,
    statistics_pre_estop,
    statistics_post_estop,
    face,
):


    # graphics of forces before and after Estop push
    font = {
        "family": "serif",
        "color": "black",
        "weight": "normal",
        "size": 10,
    }

    hatF = "$\overline{F}), N$"

    

    fig = plt.figure()
    fig.subplots_adjust(hspace=0.9, wspace=0.9)

    # line of Estop
    x_eStop = np.ones((5,), dtype=int)
    x_eStop = x_eStop * eStop
    y_eStop = np.array([-5000, -150, -100, -50, 5000])

    # line force command
    x_force = np.ones((5,), dtype=int)
    x_force = x_force * position_reached_index_force
    y_force = np.array([-5000, -150, -100, -50, 5000])

    n = 6
    for idx in range(n):
        ax = fig.add_subplot(3, 2, idx + 1)
        ax.plot(data_collected_tangent["measured"][:, idx])
        ax.plot(x_force, y_force)
        ax.plot(x_eStop, y_eStop)
        num_act = actuators_groups[idx] + 1
        ax.set_title("A%i" % (idx + 1), fontdict=font)
        ax.set_ylabel("Meas. F, N", fontdict=font)


    ax1 = plt.subplot(321)
    ax1.plot(x_force, y_force, color="orange", label="M2 cart vertical")
    ax1.plot(x_eStop, y_eStop, color="green", label="Estop")
    
    if face == 'down':
            ax1.legend(loc="upper left", fontsize="5")

    elif face == 'up':
            ax1.legend(loc="lower left", fontsize="5")

    
    if face == 'down':
        ax1.set_yticks([0, 250], ["0", "250"])
        ax1.set_ylim([-50, 250])

    elif face == 'up':
        ax1.set_yticks([-150, 0], ["-150", "0"])
        ax1.set_ylim([-150, 50])
        
    if face == 'down': 
        ax2 = plt.subplot(322)
        ax2.set_yticks([-4480, -2000, 50], ["-4480", "-2000", "50"])
        ax2.set_ylim([-4800, 50])

    elif face == 'up':
        ax2 = plt.subplot(322)
        ax2.set_yticks([50, 2000, 4480], ["50","2000", "4480"])
        ax2.set_ylim([50, 4800])

    if face == 'down':
        ax3 = plt.subplot(323)
        ax3.set_yticks([-4480, -2000, 50], ["-4480", "-2000", "50"])
        ax3.set_ylim([-4800, 50])

    if face == 'up':
        ax3 = plt.subplot(323)
        ax3.set_yticks([50, 2000, 4480], ["50", "2000", "4480"])
        ax3.set_ylim([50, 4800])

    if face == 'down':
        ax4 = plt.subplot(324)
        ax4.set_yticks([0, 250], ["0", "250"])
        ax4.set_ylim([-50, 250])

    if face == 'up':
        ax4 = plt.subplot(324)
        ax4.set_yticks([-50, 150], ["-50", "150"])
        ax4.set_ylim([-50, 150])


    if face == 'down':
        ax5 = plt.subplot(325)
        ax5.set_yticks([0, 2000, 4480], ["0", "2000", "4480"])
        ax5.set_ylim([0, 4800])

    if face == 'up':
        ax5 = plt.subplot(325)
        ax5.set_yticks([-4480, -2000, -50], ["-4480", "-2000", "-50"])
        ax5.set_ylim([-4800, -50])


    if face == 'down':
        ax6 = plt.subplot(326)
        ax6.set_yticks([0, 2000, 4480], ["0", "2000", "4480"])
        ax6.set_ylim([0, 4800])
        
    if face == 'up':
        ax6 = plt.subplot(326)
        ax6.set_yticks([-4480, -2000, -50], ["-4480", "-2000", "-50"])
        ax6.set_ylim([-4800, -50])

    
    # average forces before and after E-stop push
    fig = plt.figure()
    fig.subplots_adjust(hspace=0.9, wspace=0.9)
    hatF = "Meas. $\overline{F}, N$"

    n = 6
    for idx in range(n):
        samples_pre = data_collected_tangent["measured"][
            position_reached_index_force:eStop, idx]
        
        x_pre = np.linspace(0, len(samples_pre), num=len(samples_pre))

        samples_post = data_collected_tangent["measured"][
            eStop:full_stream, idx]
        x_post = np.linspace(0, len(samples_post), num=len(samples_post))

        rms_pre = np.ones(len(x_pre)) * statistics_pre_estop[4, idx]
        rms_post = np.ones(len(x_post)) * statistics_post_estop[4, idx]

        ax = fig.add_subplot(3, 2, idx + 1)
        ax.errorbar(x_pre, samples_pre, rms_pre, fmt="blue", ecolor="gray", lw=0.5, label="before Estop")
        ax.errorbar(x_post, samples_post, rms_post, fmt="orange", ecolor="gray", lw=0.5, label="after Estop")
        num_act = actuators_groups[idx] + 1
        ax.set_title("A%i" % (idx + 1), fontdict=font)
        ax.set_ylabel(hatF, fontdict=font)
        
        if idx == 0:
            ax.legend(loc="upper left", fontsize="5")


    fig.tight_layout()

In [ ]:
# Rigid body position from IMS after E-stop

def ims_estop(
    data_ims,
    start_time_ims,
    eStop_ims,
    group,
):
    # graphics of forces before and after Estop push
    font = {
        "family": "serif",
        "color": "black",
        "weight": "normal",
        "size": 10,
    }
    # average IMS positions before and after E-stop push

    # pre Estop, post force command

    # Mean
    samples_pre_x = pd.DataFrame(data_ims.x[start_time_ims:eStop_ims]).values.mean()
    samples_pre_y = pd.DataFrame(data_ims.y[start_time_ims:eStop_ims]).values.mean()
    samples_pre_z = pd.DataFrame(data_ims.z[start_time_ims:eStop_ims]).values.mean()
    samples_pre_xrot = pd.DataFrame(
        data_ims.xRot[start_time_ims:eStop_ims]
    ).values.mean()
    samples_pre_yrot = pd.DataFrame(
        data_ims.yRot[start_time_ims:eStop_ims]
    ).values.mean()
    samples_pre_zrot = pd.DataFrame(
        data_ims.zRot[start_time_ims:eStop_ims]
    ).values.mean()

    # RMS
    rms_x_pre_estop = pd.DataFrame(data_ims.x[start_time_ims:eStop_ims]).values.std()
    rms_y_pre_estop = pd.DataFrame(data_ims.y[start_time_ims:eStop_ims]).values.std()
    rms_z_pre_estop = pd.DataFrame(data_ims.z[start_time_ims:eStop_ims]).values.std()
    rms_xrot_pre_estop = pd.DataFrame(
        data_ims.xRot[start_time_ims:eStop_ims]
    ).values.std()
    rms_yrot_pre_estop = pd.DataFrame(
        data_ims.yRot[start_time_ims:eStop_ims]
    ).values.std()
    rms_zrot_pre_estop = pd.DataFrame(
        data_ims.zRot[start_time_ims:eStop_ims]
    ).values.std()

    # post Estop

    # Mean
    samples_post_x = pd.DataFrame(data_ims.x[eStop_ims : len(data_ims.x)]).values.mean()
    samples_post_y = pd.DataFrame(data_ims.y[eStop_ims : len(data_ims.x)]).values.mean()
    samples_post_z = pd.DataFrame(data_ims.z[eStop_ims : len(data_ims.x)]).values.mean()
    samples_post_xrot = pd.DataFrame(
        data_ims.xRot[eStop_ims : len(data_ims.x)]
    ).values.mean()
    samples_post_yrot = pd.DataFrame(
        data_ims.yRot[eStop_ims : len(data_ims.x)]
    ).values.mean()
    samples_post_zrot = pd.DataFrame(
        data_ims.zRot[eStop_ims : len(data_ims.x)]
    ).values.mean()

    # RMS
    rms_x_post_estop = pd.DataFrame(
        data_ims.x[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_y_post_estop = pd.DataFrame(
        data_ims.y[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_z_post_estop = pd.DataFrame(
        data_ims.z[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_xrot_post_estop = pd.DataFrame(
        data_ims.xRot[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_yrot_post_estop = pd.DataFrame(
        data_ims.yRot[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_zrot_post_estop = pd.DataFrame(
        data_ims.zRot[eStop_ims : len(data_ims.x)]
    ).values.std()

    ims_x = np.array([samples_pre_x, samples_post_x])
    ims_y = np.array([samples_pre_y, samples_post_y])
    ims_z = np.array([samples_pre_z, samples_post_z])
    ims_xrot = np.array([samples_pre_xrot, samples_post_xrot])
    ims_yrot = np.array([samples_pre_yrot, samples_post_yrot])
    ims_zrot = np.array([samples_pre_zrot, samples_post_zrot])

    rms_x = np.array([rms_x_pre_estop, rms_x_post_estop])
    rms_y = np.array([rms_y_pre_estop, rms_y_post_estop])
    rms_z = np.array([rms_z_pre_estop, rms_z_post_estop])
    rms_xrot = np.array([rms_xrot_pre_estop, rms_xrot_post_estop])
    rms_yrot = np.array([rms_yrot_pre_estop, rms_yrot_post_estop])
    rms_zrot = np.array([rms_zrot_pre_estop, rms_zrot_post_estop])

    x = [data_ims.index[start_time_ims], data_ims.index[eStop_ims]]

    req_linear = 0.5
    req_angular = 3e-5 * 3600 / 2

    xstr = np.array([f"{el.hour}:{el.minute}:{el.second}" for el in x])
    xstr1 = xstr[0] + " E-stop"
    xstr2 = xstr[1] + " End test"
    xstring = [xstr1, xstr2]

    fig = plt.figure()

    ax1 = plt.subplot(231)
    ax1.errorbar(x, ims_x - np.mean(ims_x), rms_x, fmt="blue", ecolor="black", lw=1.5)
    ax1.axhspan(-req_linear, req_linear, facecolor="gray", alpha=0.1)
    ax1.set_ylabel("x, \u03BCm", fontdict=font)
    ax1.set_xticks(x, xstring, rotation="vertical")

    ax2 = plt.subplot(232)
    ax2.errorbar(x, ims_y - np.mean(ims_y), rms_y, fmt="blue", ecolor="black", lw=1.5)
    ax2.axhspan(-req_linear, req_linear, facecolor="gray", alpha=0.1)
    ax2.set_ylabel("y, \u03BCm", fontdict=font)
    ax2.set_xticks(x, xstring, rotation="vertical")

    ax3 = plt.subplot(233)
    ax3.errorbar(x, ims_z - np.mean(ims_z), rms_z, fmt="blue", ecolor="black", lw=1.5)
    ax3.axhspan(-req_linear, req_linear, facecolor="gray", alpha=0.1)
    ax3.set_ylabel("z, \u03BCm", fontdict=font)
    ax3.set_xticks(x, xstring, rotation="vertical")

    ax4 = plt.subplot(234)
    ax4.errorbar(
        x, ims_xrot - np.mean(ims_xrot), rms_xrot, fmt="blue", ecolor="black", lw=1.5
    )
    ax4.axhspan(-req_angular, req_angular, facecolor="gray", alpha=0.1)
    ax4.set_ylabel("xRot, arcsec", fontdict=font)
    ax4.set_xticks(x, xstring, rotation="vertical")

    ax5 = plt.subplot(235)
    ax5.errorbar(
        x, ims_yrot - np.mean(ims_yrot), rms_yrot, fmt="blue", ecolor="black", lw=1.5
    )
    ax5.axhspan(-req_angular, req_angular, facecolor="gray", alpha=0.1)
    ax5.set_ylabel("yRot, arcsec", fontdict=font)
    ax5.set_xticks(x, xstring, rotation="vertical")

    ax6 = plt.subplot(236)
    ax6.errorbar(
        x, ims_zrot - np.mean(ims_zrot), rms_zrot, fmt="blue", ecolor="black", lw=1.5
    )
    ax6.axhspan(-req_angular, req_angular, facecolor="gray", alpha=0.1)
    ax6.set_ylabel("zRot, arcsec", fontdict=font)
    ax6.set_xticks(x, xstring, rotation="vertical")

    fig.tight_layout()

    ims = {
        'x': [(ims, rms) for ims, rms in zip(ims_x, rms_x)],
        'y': [(ims, rms) for ims, rms in zip(ims_y, rms_y)],
        'z': [(ims, rms) for ims, rms in zip(ims_z, rms_z)],
        'xRot': [(ims, rms) for ims, rms in zip(ims_xrot, rms_xrot)],
        'yRot': [(ims, rms) for ims, rms in zip(ims_yrot, rms_yrot)],
        'zRot': [(ims, rms) for ims, rms in zip(ims_zrot, rms_zrot)]
    }

    return ims

    

In [ ]:
# plot and fit tangent actuator encoder values

async def plot_encoder(
    efd_client,
    group,
    actuators_group,
    time_start,
    time_end,
    time_estop,
    start_time_encoder,
):
    topic = "lsst.sal.MTM2.tangentEncoderPositions"
    topic_fields = await efd_client.get_fields(topic)

    query = efd_client.build_time_range_query(topic, topic_fields, time_start, time_end)
    query_df = await efd_client.influx_client.query(query)

    font = {
        "family": "serif",
        "color": "black",
        "weight": "normal",
        "size": 10,
    }

    fig, axs = plt.subplots(nrows=3, ncols=2)
    c = 0
    j = 0
    for i, el in enumerate(actuators_group):
        axs[j, c].plot(query_df.index, query_df[f"position{i}"])
        axs[j, c].axvline(time_estop, color="green", label="Estop")
        axs[j, c].axvline(start_time_encoder, color="orange", label="M2 vertical position")

        axs[j, c].set_title("A%i" % (i + 1), fontdict=font)
        if i == 2:
            axs[j, c].legend(loc="lower left", fontsize="5")
        #axs[j, c].legend(loc="lower left", fontsize="5")
        axs[j, c].set_xlabel("Time", fontdict=font)
        axs[j, c].set_ylabel("Encoder, \u03BCm", fontdict=font)
        axs[j, c].get_xaxis().set_ticks([])

        
        
        c += 1
        if c > 1:
            j += 1
            c = 0
            
    fig.tight_layout()

    

    # linear fit encoder vs. time
    fig, axs = plt.subplots(nrows=3, ncols=2)
    c = 0
    j = 0

    mlist = list()

    for i, el in enumerate(actuators_group):
        encoder_drift = query_df[f"position{i}"][query_df.index > time_estop]
        x_post = np.linspace(0, len(encoder_drift), num=len(encoder_drift))
        y_post = encoder_drift.iloc[:].values
        coef = np.polyfit(x_post, y_post, 1)
        mlist.append((i, coef[0]))
        poly1d_fn = np.poly1d(coef)

        axs[j, c].plot(x_post, y_post, "yo", x_post, poly1d_fn(x_post), "--k")
        axs[j, c].set_title(f"A{i+1}, m={coef[0]:.2e}", fontdict=font)
        axs[j, c].set_xlabel("Time", fontdict=font)
        axs[j, c].set_ylabel("Encoder, \u03BCm", fontdict=font)
        axs[j, c].get_xaxis().set_ticks([])
        axs[j, c].get_yaxis().set_ticks([-2000, 2000])

        c += 1
        if c > 1:
            j += 1
            c = 0

    fig.tight_layout()

    return mlist

## Instantiate the MockControlClosedLoop

In [ ]:
config_path = Path.home() / "WORK" / "ts_mtm2" / "config" / "parameter_files"
filepath_cell_geom = Path.home() / "WORK" / "ts_mtm2" / "config" / "cellGeom.json"

In [ ]:
control_closed_loop = MockControlClosedLoop()
control_closed_loop.load_file_cell_geometry(filepath_cell_geom)

## Instantiate the DiagnosticsM2 Class

Notice that the UTC time is used when doing the query.

In [ ]:
diagnostics_m2 = DiagnosticsM2(efd_name=EfdName.Idf)

# Instatiate EFD client

In [ ]:
efd_client = lsst_efd_client.EfdClient(efd_name="idf_efd")

## Tangent actuators

In [ ]:
actuators_groups = np.array([72, 73, 74, 75, 76, 77])

## M2 vertical, weight load on A3-A5

In [ ]:
groups_downward = {
    0: [
        Time("2023-05-24T14:00:00", scale="utc", format="isot"),
        Time("2023-05-24T14:08:00", scale="utc", format="isot"),
    ],
}

In [ ]:
m_encoder = {
    f'group{i}': {'downward': None, 'upward': None} for i in range(1)
}

In [ ]:
# run analysis for the M2 cart standing vertical on A3-A5 actuators

print("************************** \n MIRROR vertical on A3-A5 \n**************************")
for key, val in groups_downward.items():
    time_start = val[0]
    time_end = val[1]

    (
        xy_actuators,
        data_ims, 
        data_collected_axial, 
        data_collected_tangent, 
        data_power_status, 
        data_inclinometer
    ) = await query_data(diagnostics_m2, control_closed_loop, time_start, time_end)

    
    (
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        eStop,
        eStop_ims,
        time_estop,
        position_reached_index_force,
        start_time_ims,
        start_time_encoder,
    ) = print_data(
        data_collected_axial,
        data_collected_tangent,
        data_inclinometer[0],
        data_ims,
        data_power_status,
        actuators_groups,
        key,
        "down",
    )

    actuator_plot(
        diagnostics_m2,
        control_closed_loop,
        position_reached_index_force,
        actuators_groups,
        key,
        eStop,
        data_collected_axial,
        data_collected_tangent,
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        "down",
    )

    ims = ims_estop(
        data_ims,
        start_time_ims,
        eStop_ims,
        key,
    )

    m_encoder[f'group{key}']['downward'] = await plot_encoder(
            efd_client,
            key,
            actuators_groups,
            time_start,
            time_end,
            time_estop,
            start_time_encoder,
        )

    with open("ims_table.txt", "a") as ims_tab_file:
        if key == 0:
            ims_tab_file.write("*** MIRROR vertical on A3-A5 ***\n")
        else:
            ims_tab_file.write("\n\n")

        ims_tab_file.write(f"***\nACTUATOR {key+1}\n")
        ims_tab_file.write(
            tabulate.tabulate(
                [(f'{ims[0][0]:.2f} +/- {ims[0][1]:.2f}', f'{ims[1][0]:.2} +/- {ims[1][1]:.2f}') for key, ims in ims.items() if key != "time"],
                headers=["DoF", "Command Time", "eStop"],
                showindex=list(ims.keys()),
                tablefmt="rst",
            )
        )


## M2 vertical, weight load on A2-A6

In [ ]:
groups_upward = {
    0: [
        Time("2023-05-24T15:29:30", scale="utc", format="isot"),
        Time("2023-05-24T15:38:30", scale="utc", format="isot"),
    ],        
}

In [ ]:
# run analysis for the M2 cart standing vertical on A2-A6 actuators

print("************************** \n MIRROR vertical on A2-A6 \n**************************")
for key, val in groups_upward.items():
    time_start = val[0]
    time_end = val[1]

    (
        xy_actuators,
        data_ims, 
        data_collected_axial, 
        data_collected_tangent, 
        data_power_status, 
        data_inclinometer
    ) = await query_data(diagnostics_m2, control_closed_loop, time_start, time_end)

    
    (
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        eStop,
        eStop_ims,
        time_estop,
        position_reached_index_force,
        start_time_ims,
        start_time_encoder,
    ) = print_data(
        data_collected_axial,
        data_collected_tangent,
        data_inclinometer[0],
        data_ims,
        data_power_status,
        actuators_groups,
        key,
        "up",
    )

    actuator_plot(
        diagnostics_m2,
        control_closed_loop,
        position_reached_index_force,
        actuators_groups,
        key,
        eStop,
        data_collected_axial,
        data_collected_tangent,
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        "up",
    )

    ims = ims_estop(
        data_ims,
        start_time_ims,
        eStop_ims,
        key,
    )


    m_encoder[f'group{key}']['upward'] = await plot_encoder(
            efd_client,
            key,
            actuators_groups,
            time_start,
            time_end,
            time_estop,
            start_time_encoder,
        )

    with open("ims_table.txt", "a") as ims_tab_file:
        if key == 0:
            ims_tab_file.write("\n\n*** MIRROR vertical on A2-A6 ***")
        else:
            ims_tab_file.write("\n\n")

        ims_tab_file.write(f"***\nACTUATOR {key+1}\n")
        ims_tab_file.write(
            tabulate.tabulate(
                [(f'{ims[0][0]:.2f} +/- {ims[0][1]:.2f}', f'{ims[1][0]:.2} +/- {ims[1][1]:.2f}') for key, ims in ims.items() if key != "time"],
                headers=["DoF", "Command Time", "eStop"],
                showindex=list(ims.keys()),
                tablefmt="rst",
            )
        )


In [ ]:

# create a table of the angular coefficient from the encoder values linear 

with open("m_encoder_table.txt", "a") as menc_tab_file:
    for i, group in enumerate(m_encoder):
        menc_tab_file.write(f'*** ACTUATOR GROUP {i+1}***\n')
        menc_tab_file.write(
            tabulate.tabulate(
                [
                    (f'{el_up[1]:.1e}', f'{el_down[1]:.1e}')
                    for el_up, el_down in zip(*m_encoder[group].values())
                ],
                headers=["Id. Actuator", "Downdard", "Upward"],
                showindex=[el[0] for el in m_encoder[f"group{i}"]["downward"]],
                tablefmt="rst",
            )
        )
        menc_tab_file.write('\n')